In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
# 检查 GPU 是否可用  
import torch
if torch.cuda.is_available():  
   print("GPU 可用")  
   # 获取 GPU 设备数量  
   num_gpus = torch.cuda.device_count()  
   print(f"共有{num_gpus}个 GPU 设备")
   # 获取 GPU 设备信息  
   for i in range(num_gpus):  
       print(f"设备{i}: {torch.cuda.get_device_properties(i)}")  
else:  
   print("GPU 不可用")  
import sys
# sys.path.append("/workspace/notebooks/")
# sys.path.append("/workspace/notebooks/FSS/")
import numpy as np
import matplotlib.pyplot as plt

GPU 可用
共有1个 GPU 设备
设备0: _CudaDeviceProperties(name='NVIDIA GeForce RTX 2080 Ti', major=7, minor=5, total_memory=11011MB, multi_processor_count=68)


In [2]:
import sys
sys.path.append("../packages/autoqnn/")
import autoqnn
from dns_msdnet import msdnet_cifar100, msdnet_imagenet

# DNS MSDNet 
测试DNS在MSDNet上的效果如何，首先是在Cifar100数据集上的效果，先测试一下模型

In [3]:
model = msdnet_cifar100().cuda()
model_name = "msdnet_cifar100"

building network of steps: 
[4, 2, 2, 2, 2, 2, 2] 16
 ********************** Block 1  **********************
|		inScales 3 outScales 3 inChannels 16 outChannels 16		|

|		inScales 3 outScales 3 inChannels 32 outChannels 16		|

|		inScales 3 outScales 3 inChannels 48 outChannels 16		|

|		inScales 3 outScales 3 inChannels 64 outChannels 16		|

 ********************** Block 2  **********************
|		inScales 3 outScales 3 inChannels 80 outChannels 16		|

|		inScales 3 outScales 3 inChannels 96 outChannels 16		|

 ********************** Block 3  **********************
|		inScales 3 outScales 2 inChannels 112 outChannels 16		|
|		Transition layer inserted! (max), inChannels 128, outChannels 64	|

|		inScales 2 outScales 2 inChannels 64 outChannels 16		|

 ********************** Block 4  **********************
|		inScales 2 outScales 2 inChannels 80 outChannels 16		|

|		inScales 2 outScales 2 inChannels 96 outChannels 16		|

 ********************** Block 5  **********************
|		inS

In [4]:
# model

In [5]:
# get dataset
from autoqnn import datasets
torch_weights_path=f"{os.environ['HOME']}/models/torch/weights"
data_root="~/datasets/cifar100"
dataset_name = 'cifar100'
trainloader,testloader,classes = datasets.cifar.get_cifar_dataloader(
    root=data_root,dataset=dataset_name,autoaugment=True)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
from torch import nn
import time
import shutil

class KDLoss(nn.Module):
    def __init__(self, temperature, gamma, nBlocks):
        super(KDLoss, self).__init__()
        
        self.kld_loss = nn.KLDivLoss().cuda()
        self.ce_loss = nn.CrossEntropyLoss().cuda()
        self.log_softmax = nn.LogSoftmax(dim=1).cuda()
        self.softmax = nn.Softmax(dim=1).cuda()

        self.T = temperature
        self.gamma = gamma
        self.nBlocks = nBlocks

    def loss_fn_kd(self, outputs, targets, soft_targets):
        loss = self.ce_loss(outputs[-1], targets)
        T = self.T
        for i in range(self.nBlocks - 1):
            _ce = (1. - self.gamma) * self.ce_loss(outputs[i], targets)
            _kld = self.kld_loss(self.log_softmax(outputs[i] / T), self.softmax(soft_targets.detach() / T)) * self.gamma * T * T
            loss = loss + _ce + _kld
        return loss

def train(train_loader, model, kd_loss, optimizer, epoch, epochs, init_lr, print_freq=10):

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1, top5 = [], []
    for i in range(model.nBlocks):
        top1.append(AverageMeter())
        top5.append(AverageMeter())

    # switch to train mode
    model.train()

    end = time.time()

    running_lr = None
    for i, (input, target) in enumerate(train_loader):

        lr = adjust_learning_rate(optimizer, epoch, epochs, init_lr, batch=i,
                                  nBatch=len(train_loader), method="multistep")
        # measure data loading time
        if running_lr is None:
            running_lr = lr

        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output, middle_feas = model(input_var)
        if not isinstance(output, list):
            output = [output]

        loss = kd_loss.loss_fn_kd(output, target_var, output[-1])
        losses.update(loss.item(), input.size(0))
        for j in range(len(output)):
            acc1, acc5 = accuracy(output[j].data, target, topk=(1, 5))
            top1[j].update(acc1.item(), input.size(0))
            top5[j].update(acc5.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.avg:.3f}\t'
                  'Data {data_time.avg:.3f}\t'
                  'Loss {loss.val:.4f}\t'
                  'Acc@1 {top1.val:.4f}\t'
                  'Acc@5 {top5.val:.4f}'.format(
                    epoch, i + 1, len(train_loader),
                    batch_time=batch_time, data_time=data_time,
                    loss=losses, top1=top1[-1], top5=top5[-1]))

    return losses.avg, top1[-1].avg, top5[-1].avg, running_lr

def validate(val_loader, model, kd_loss,print_freq=10):
    batch_time = AverageMeter()
    losses = AverageMeter()
    data_time = AverageMeter()
    top1, top5 = [], []
    for i in range(model.nBlocks):
        top1.append(AverageMeter())
        top5.append(AverageMeter())

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            target = target.cuda()
            input = input.cuda()

            input_var = torch.autograd.Variable(input)
            target_var = torch.autograd.Variable(target)

            data_time.update(time.time() - end)

            # compute output
            output,_ = model(input_var)
            if not isinstance(output, list):
                output = [output]

            loss = kd_loss.loss_fn_kd(output, target_var, output[-1])

            # measure error and record loss
            losses.update(loss.item(), input.size(0))

            for j in range(len(output)):
                acc1, acc5 = accuracy(output[j].data, target, topk=(1, 5))
                top1[j].update(acc1.item(), input.size(0))
                top5[j].update(acc5.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:
                print('Epoch: [{0}/{1}]\t'
                      'Time {batch_time.avg:.3f}\t'
                      'Data {data_time.avg:.3f}\t'
                      'Loss {loss.val:.4f}\t'
                      'Acc@1 {top1.val:.4f}\t'
                      'Acc@5 {top5.val:.4f}'.format(
                        i + 1, len(val_loader),
                        batch_time=batch_time, data_time=data_time,
                        loss=losses, top1=top1[-1], top5=top5[-1]))
                # break
    for j in range(model.nBlocks):
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'.format(top1=top1[j], top5=top5[j]))
        """
        print('Exit {}\t'
              'Err@1 {:.4f}\t'
              'Err@5 {:.4f}'.format(
              j, top1[j].avg, top5[j].avg))
        """
    # print(' * Err@1 {top1.avg:.3f} Err@5 {top5.avg:.3f}'.format(top1=top1[-1], top5=top5[-1]))
    return losses.avg, top1[-1].avg, top5[-1].avg

def save_checkpoint(state, save_path, is_best, filename, result):
    print(save_path)
    result_filename = os.path.join(save_path, 'scores.tsv')
    model_dir = os.path.join(save_path, 'save_models')
    latest_filename = os.path.join(model_dir, 'latest.txt')
    model_filename = os.path.join(model_dir, filename)
    best_filename = os.path.join(model_dir, 'model_best.pth.tar')
    os.makedirs(save_path, exist_ok=True)
    os.makedirs(model_dir, exist_ok=True)
    print("=> saving checkpoint '{}'".format(model_filename))

    torch.save(state, model_filename)

    with open(result_filename, 'w') as f:
        print('\n'.join(result), file=f)

    with open(latest_filename, 'w') as fout:
        fout.write(model_filename)
    if is_best:
        shutil.copyfile(model_filename, best_filename)

    print("=> saved checkpoint '{}'".format(model_filename))
    return

def load_checkpoint(save_path):
    model_dir = save_path
    latest_filename = os.path.join(model_dir, 'latest.txt')
    if os.path.exists(latest_filename):
        with open(latest_filename, 'r') as fin:
            model_filename = fin.readlines()[0]
    else:
        return None
    print("=> loading checkpoint '{}'".format(model_filename))
    state = torch.load(model_filename)
    print("=> loaded checkpoint '{}'".format(model_filename))
    return state

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the error@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        # res.append(100.0 - correct_k.mul_(100.0 / batch_size))
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

import math
def adjust_learning_rate(optimizer, epoch, epochs, init_lr, batch=None,
                         nBatch=None, method='multistep'):
    if method == 'cosine':
        T_total = epochs * nBatch
        T_cur = (epoch % epochs) * nBatch + batch
        lr = 0.5 * init_lr * (1 + math.cos(math.pi * T_cur / T_total))
    elif method == 'multistep':
        data = "cifar100"
        if data.startswith('cifar'):
            lr, decay_rate = init_lr, 0.1
            if epoch >= epochs * 0.75:
                lr *= decay_rate ** 2
            elif epoch >= epochs * 0.5:
                lr *= decay_rate
        else:
            lr = init_lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

In [7]:
# training code
init_lr = 0.1
momentum = 0.9
weight_decay = 5e-4
epochs = 200
temperature = 3.0
gamma = 0.9
nBlocks = 7
optimizer = torch.optim.SGD(model.parameters(),
                                init_lr,
                                momentum=momentum,
                                weight_decay=weight_decay)
kd_loss = KDLoss(temperature, gamma, nBlocks)

In [ ]:
scores = ['epoch\tlr\ttrain_loss\tval_loss\ttrain_acc1'
              '\tval_acc1\ttrain_acc5\tval_acc5']

import sys
import time
# 将程序的输出重定向到文件  
file_path = f'./training_logs/{model_name}-on-cifar100-lr-{init_lr}-wd-{weight_decay}-epoch-{epochs}.txt'  
sys.stdout = open(file_path, 'w')
best_acc1 = 0.0
for epoch in range(epochs):
    
    # train for one epoch
    train_loss, train_acc1, train_acc5, lr = train(trainloader, model, kd_loss, optimizer, epoch,epochs,init_lr)

    # evaluate on validation set
    val_loss, val_acc1, val_acc5 = validate(testloader, model, kd_loss)

    # save scores to a tsv file, rewrite the whole file to prevent
    # accidental deletion
    scores.append(('{}\t{:.3f}' + '\t{:.4f}' * 6)
                  .format(epoch, lr, train_loss, val_loss,
                          train_acc1, val_acc1, train_acc5, val_acc5))

    is_best = val_acc1 > best_acc1
    if is_best:
        best_acc1 = val_acc1
        best_epoch = epoch
        print('Best var_acc1 {}'.format(best_acc1))

    save_path = f"./models/{model_name}-cifar100/"
    model_filename = 'checkpoint_%03d.pth.tar' % epoch
    save_checkpoint({
        'epoch': epoch,
        'arch': model_name,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer': optimizer.state_dict(),
    }, save_path, is_best, model_filename, scores)

print('Best val_acc1: {:.4f} at epoch {}'.format(best_acc1, best_epoch))

# Save the trained parameters to disk
# best_loss = np.min(val_loss)
# best_metric = np.max(val_metrics）
save_file_name = f"./models/{model_name}-cifar100-top1-{best_acc1}.pth"
torch.save(model.state_dict(),save_file_name)
print(f"save model weight to {save_file_name}")
# 关闭文件  
sys.stdout.close()  

In [13]:
train(trainloader, model, kd_loss, optimizer, 1,epochs,init_lr)

Epoch: [1][1/100]	Time 1.063	Data 0.907	Loss 6.1577	Acc@1 13.2000	Acc@5 36.6000
Epoch: [1][11/100]	Time 0.391	Data 0.086	Loss 6.1149	Acc@1 13.6000	Acc@5 37.6000
Epoch: [1][21/100]	Time 0.364	Data 0.047	Loss 6.1194	Acc@1 13.4000	Acc@5 40.4000
Epoch: [1][31/100]	Time 0.356	Data 0.033	Loss 6.1191	Acc@1 14.8000	Acc@5 37.2000
Epoch: [1][41/100]	Time 0.353	Data 0.026	Loss 6.0268	Acc@1 16.6000	Acc@5 41.8000
Epoch: [1][51/100]	Time 0.356	Data 0.022	Loss 6.0687	Acc@1 12.8000	Acc@5 35.8000
Epoch: [1][61/100]	Time 0.376	Data 0.019	Loss 5.9661	Acc@1 16.8000	Acc@5 42.0000
Epoch: [1][71/100]	Time 0.397	Data 0.017	Loss 5.9588	Acc@1 16.6000	Acc@5 42.2000
Epoch: [1][81/100]	Time 0.413	Data 0.015	Loss 5.9773	Acc@1 15.0000	Acc@5 37.8000
Epoch: [1][91/100]	Time 0.425	Data 0.014	Loss 6.0855	Acc@1 14.2000	Acc@5 36.8000


(6.0535579013824465, 14.726000289916993, 38.98800092697144, 0.1)